In [1]:
# -*- coding: utf-8 -*-
"""
predict.py

Carga un modelo Transformer entrenado con SentencePiece, 
recibe un contexto de entrada y genera texto adicional 
en árabe (u otro idioma), guardando el resultado en un 
archivo de texto UTF-8 con un prefijo \u200f para facilitar 
la lectura en RTL.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import sentencepiece as spm  
import math

In [2]:
def build_rope_sin_cos(head_dim, block_size, base=10000):
    """
    Construye tensores de seno y coseno para posiciones [0, block_size-1],
    asumiendo que 'head_dim' es la dimensión de cada cabeza (debe ser par).

    Args:
        head_dim (int): Dimensión de cada cabeza (por ejemplo, 64).
        block_size (int): Longitud máxima de la secuencia.
        base (int): Base para la escala de frecuencia (usualmente 10000).

    Returns:
        sin (Tensor): Tensor de senos de forma (1, block_size, head_dim//2).
        cos (Tensor): Tensor de cosenos de forma (1, block_size, head_dim//2).
    """
    half_dim = head_dim // 2
    # Calcular frecuencias para cada componente de la mitad
    freq_seq = torch.arange(0, half_dim, dtype=torch.float32)
    freq_seq = base ** (2 * freq_seq / (head_dim * 2))  # Escala según la dimensión

    positions = torch.arange(block_size, dtype=torch.float32).unsqueeze(1)  # (block_size, 1)
    div_term = positions / freq_seq.unsqueeze(0)  # (block_size, half_dim)
    
    sin = torch.sin(div_term).unsqueeze(0)  # (1, block_size, half_dim)
    cos = torch.cos(div_term).unsqueeze(0)  # (1, block_size, half_dim)
    return sin, cos

def apply_rope(q, k, rope_sin, rope_cos):
    """
    Aplica Rotary Positional Embeddings a los tensores de consulta y clave
    de un único head.

    Args:
        q (Tensor): Tensor de consulta de forma (B, T, head_dim).
        k (Tensor): Tensor de clave de forma (B, T, head_dim).
        rope_sin (Tensor): Tensores de seno, de forma (1, block_size, head_dim//2).
        rope_cos (Tensor): Tensores de coseno, de forma (1, block_size, head_dim//2).

    Returns:
        q_final, k_final (Tensors): Tensores rotados, de forma (B, T, head_dim).
    """
    B, T, C = q.shape  # C = head_dim
    half_dim = C // 2

    # Dividir q y k en dos mitades
    q_left, q_right = q[..., :half_dim], q[..., half_dim:]
    k_left, k_right = k[..., :half_dim], k[..., half_dim:]
    
    # Seleccionar las posiciones (hasta T) y expandir para tener (B, T, half_dim)
    sin_t = rope_sin[:, :T, :].expand(B, T, half_dim)
    cos_t = rope_cos[:, :T, :].expand(B, T, half_dim)
    
    # Aplicar la rotación: 
    # q_rot = q_left * cos - q_right * sin
    # q_pass = q_left * sin + q_right * cos
    q_rot = q_left * cos_t - q_right * sin_t
    q_pass = q_left * sin_t + q_right * cos_t
    q_final = torch.cat((q_rot, q_pass), dim=-1)
    
    k_rot = k_left * cos_t - k_right * sin_t
    k_pass = k_left * sin_t + k_right * cos_t
    k_final = torch.cat((k_rot, k_pass), dim=-1)
    
    return q_final, k_final

In [3]:
# -------------------------------------------------------------------------------
# ENCABEZADO DEL MODELO
# -------------------------------------------------------------------------------
class Head(nn.Module):
    """
    Cabeza de self-attention con máscara causal y aplicación de RoPE.
    
    Esta implementación es para un único head; se espera que la dimensión
    de entrada 'n_embd' en este módulo corresponda a 'head_dim' (por ejemplo, 64).
    """
    def __init__(self, n_embd, head_size, block_size, dropout):
        """
        Args:
            n_embd (int): Dimensión de entrada (debe ser igual a head_size para este head).
            head_size (int): Tamaño de cada cabeza (por ejemplo, 64).
            block_size (int): Longitud máxima de la secuencia.
            dropout (float): Tasa de dropout.
        """
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        
        # Máscara causal: para que cada token no vea tokens futuros.
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, rope_sin, rope_cos):
        """
        Args:
            x (Tensor): Entrada de forma (B, T, head_dim), donde head_dim = head_size.
            rope_sin, rope_cos (Tensors): Tensores de RoPE con forma (1, block_size, head_dim//2).
        
        Returns:
            out (Tensor): Salida de la cabeza, de forma (B, T, head_size).
        """
        B, T, C = x.shape
        q = self.query(x)  # (B, T, head_size)
        k = self.key(x)
        v = self.value(x)
        
        # Aplicar RoPE a q y k
        q_rot, k_rot = apply_rope(q, k, rope_sin, rope_cos)
        
        # Calcular atención escalada
        scale = C ** -0.5
        wei = q_rot @ k_rot.transpose(-2, -1) * scale  # (B, T, T)
        
        # Aplicar máscara causal
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        # Combinar la atención con los valores
        out = wei @ v  # (B, T, head_size)
        return out

# -------------------------------------------------
# BLOQUES DEL TRANSFORMER (con FeedForward usando GELU)
# -------------------------------------------------
class MultiHeadAttention(nn.Module):
    """
    Ejemplo de MultiHeadAttention (sin details de RoPE, 
    pero podrías integrarlo aquí).
    """
    def __init__(self, n_embd, n_heads, block_size, dropout):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = n_embd // n_heads
        self.block_size = block_size

        self.wq = nn.Linear(n_embd, n_embd, bias=False)
        self.wk = nn.Linear(n_embd, n_embd, bias=False)
        self.wv = nn.Linear(n_embd, n_embd, bias=False)
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

        # Máscara causal
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape

        # Proyecciones
        q_full = self.wq(x)  # (B, T, n_embd)
        k_full = self.wk(x)
        v_full = self.wv(x)

        # Dividir heads
        q = q_full.view(B, T, self.n_heads, self.head_dim).permute(0,2,1,3)
        k = k_full.view(B, T, self.n_heads, self.head_dim).permute(0,2,1,3)
        v = v_full.view(B, T, self.n_heads, self.head_dim).permute(0,2,1,3)

        # Atención
        wei = q @ k.transpose(-2, -1) * (self.head_dim**-0.5)
        mask = self.tril[:T, :T]
        wei = wei.masked_fill(mask==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        out = wei @ v  # (B, n_heads, T, head_dim)
        # Reordenar
        out = out.permute(0,2,1,3).contiguous()
        out = out.view(B, T, self.n_heads*self.head_dim)
        # Proyección final
        out = self.proj(out)
        out = self.dropout(out)
        return out


class FeedForward(nn.Module):
    """
    MLP tras la atención, usando GELU en lugar de ReLU.
    """
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.GELU(),           # Cambiamos ReLU -> GELU
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """
    Bloque Transformer con:
    - MultiHeadAttention
    - FeedForward (con GELU)
    - Residual connections
    """
    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.sa = MultiHeadAttention(n_embd, n_head, block_size, dropout)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ffwd = FeedForward(n_embd, dropout)

    def forward(self, x):
        # Atención
        x_ln = self.ln1(x)
        attn_out = self.sa(x_ln)
        x = x + attn_out
        # FF
        x_ln2 = self.ln2(x)
        ff_out = self.ffwd(x_ln2)
        x = x + ff_out
        return x


class BigramLanguageModel(nn.Module):
    """
    Modelo de lenguaje con:
    - Embeddings
    - Bloques (Multi-Head + FeedForward con GELU)
    - Funciones de generación con top-k / top-p / temperatura
    """
    def __init__(self, vocab_size, block_size, n_embd, n_head, n_layer, dropout=0.2):
        super().__init__()
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.n_embd = n_embd
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)

        # Pila de bloques
        self.blocks = nn.ModuleList([
            Block(n_embd, n_head, block_size, dropout) 
            for _ in range(n_layer)
        ])

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        if T > self.block_size:
            raise ValueError("Secuencia excede block_size")

        x = self.token_embedding_table(idx)  # (B, T, n_embd)

        for block in self.blocks:
            x = block(x)

        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(
        self, 
        idx, 
        max_new_tokens: int, 
        temperature: float=1.0, 
        top_k: int=None, 
        top_p: float=None
    ):
        """
        Genera tokens con top-k / top-p sampling y temperatura.
        idx: (B, T) secuencia de tokens
        max_new_tokens: cuántos tokens generar
        temperature: escalado de logits
        top_k: mantiene sólo los k más probables
        top_p: nucleus sampling, mantiene sólo tokens cumul>= top_p
        """
        for _ in range(max_new_tokens):
            # Cortar contexto
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            # Tomar logits del último token
            logits = logits[:, -1, :]  # (B, vocab_size)

            # Ajustar por temperatura
            if temperature != 1.0:
                logits = logits / temperature

            # Convertir a probabilidades
            probs = F.softmax(logits, dim=-1)  # (B, vocab_size)

            # top-k
            if (top_k is not None) and (top_k > 0):
                # Coger los k mejores
                values, indices = torch.topk(probs, top_k, dim=-1)
                probs_topk = torch.zeros_like(probs).scatter_(-1, indices, values)
                probs_topk = probs_topk / probs_topk.sum(dim=-1, keepdim=True)
                probs = probs_topk

            # top-p (nucleus sampling)
            if (top_p is not None) and (0.0 < top_p < 1.0):
                sorted_probs, sorted_indices = torch.sort(probs, descending=True, dim=-1)
                cumulative = torch.cumsum(sorted_probs, dim=-1)
                # máscara booleana: qué tokens entran en el top_p
                cutoff = (cumulative > top_p).float()
                # shift para que el primero que supere top_p se incluya
                cutoff = torch.roll(cutoff, 1, dims=-1)
                cutoff[..., 0] = 0
                # poner 0 en los tokens que exceden top_p
                sorted_probs[cutoff.bool()] = 0.0
                # renormalizar
                sorted_probs = sorted_probs / sorted_probs.sum(dim=-1, keepdim=True)
                # re-esparcir
                probs_top_p = torch.zeros_like(probs).scatter_(-1, sorted_indices, sorted_probs)
                probs = probs_top_p

            # muestrear
            idx_next = torch.multinomial(probs, num_samples=1)
            # concatenar
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [4]:
# -------------------------------------------------------------------------------
# CARGAR CHECKPOINT (con RoPE)
# -------------------------------------------------------------------------------
# Carga del checkpoint
checkpoint_path = 'bigram_model_checkpoint.pt'
checkpoint = torch.load(checkpoint_path, map_location='cpu')

# Extraer los hiperparámetros guardados (o usar los que ya tengas definidos)
vocab_size = checkpoint['vocab_size']
block_size = checkpoint['block_size']
n_embd = checkpoint['n_embd']
n_head = checkpoint['n_head']
n_layer = checkpoint['n_layer']

# Instanciar el modelo con los mismos parámetros
model = BigramLanguageModel(
    vocab_size=vocab_size,
    block_size=block_size,
    n_embd=n_embd,
    n_head=n_head,
    n_layer=n_layer,
    dropout=0.2  # o el valor que usaste en train
)

# Cargar los pesos
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Mover a GPU si hay
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


BigramLanguageModel(
  (token_embedding_table): Embedding(9500, 384)
  (blocks): ModuleList(
    (0-3): 4 x Block(
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (sa): MultiHeadAttention(
        (wq): Linear(in_features=384, out_features=384, bias=False)
        (wk): Linear(in_features=384, out_features=384, bias=False)
        (wv): Linear(in_features=384, out_features=384, bias=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
    )
  )
  (ln_f): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(

In [5]:
#-------------------------------------------------------------------------------
# CARGAR el modelo de SentencePiece usado para tokenizar
# -------------------------------------------------------------------------------
# Si guardaste la ruta del modelo en el checkpoint, por ejemplo en 'sp_model_file':
# sp_model_file = checkpoint['sp_model_file']
# Si no, asume que se llama "arabic_spm.model".
sp_model_file = "arabic_spm.model"

sp = spm.SentencePieceProcessor()
sp.load(sp_model_file)

# Definimos funciones de encode/decode (Subword)
def encode_text(text_str: str):
    """Convierte un string en subwords IDs usando SentencePiece."""
    return sp.encode(text_str, out_type=int)

def decode_tokens(token_list: list):
    """Convierte una lista de subword IDs a texto."""
    return sp.decode(token_list)


In [14]:
# -------------------------------------------------------------------------------
# RECIBIR CONTEXTO Y GENERAR TEXTO
# -------------------------------------------------------------------------------
# Ejemplo de contexto
# context_input = "بسم الله الرحمن الرحيم ياأيمن"
context_input = "بسم الله الرحمن الرحيم يا عثمان يا صانع الذكاء الاصطناعي "
context_encoded = encode_text(context_input)
context_tensor = torch.tensor([context_encoded], dtype=torch.long, device=device)

max_new_tokens = 70
generated_tensor = model.generate(context_tensor, max_new_tokens=max_new_tokens)

generated_ids = generated_tensor[0].tolist()
generated_text = decode_tokens(generated_ids)

arabic_text = "\u200f" + generated_text
print("Texto generado:\n", arabic_text)

with open('resultado_prediccion.txt', 'w', encoding='utf-8') as f:
    f.write(arabic_text)

print("Resultado guardado en 'resultado_prediccion.txt'.")

Texto generado:
 ‏بسم الله الرحمن الرحيم يا عثمان يا صانع الذكاء الاصطناعيۭا ينصرون تس أو بفٰحش معكم ٱلذىٓ نزل من ٱلحق ثم ٱلطيرهم بهۦ ۗ وكان ٱلله ٱلقايٰت ٱلله ترجعون لهم أهيمۭ وي بهۦٰه ۚ إنهۥةۢ للذين ٱلرحمٰن فسوفطط متٰعۭا بهۦ ٱليقين ٱل ٱلرشد فى كانوا۟ ٱلقوم يقٰتلوكم يستلقونيهم سيضۭ من قلوبكم ۚ ٱلق ٱلعظيم ۖ فٱذكرون بٱلحق ۚ وما كان ٱدخلوا۟ إذآ ءامنوا۟ ٱشرۢلت من تلك ٱرتضىٰ
Resultado guardado en 'resultado_prediccion.txt'.
